In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Towns/Cities Urls That I would like to scrap from the stats Canada
urls = [
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=cobourg&DGUIDlist=2021A00053514021&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=port%20hope&DGUIDlist=2021A00053514020&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=belleville&DGUIDlist=2021A00053512005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Kingston&DGUIDlist=2021A00053510010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Peterborough&DGUIDlist=2021A00053515014&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Lindsay&DGUIDlist=2021S05100472&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Sudbury&DGUIDlist=2021A00053553005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Ottawa&DGUIDlist=2021A00053506008&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=orillia&DGUIDlist=2021A00053543052&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Niagara%20falls&DGUIDlist=2021A00053526043&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=st%2E%20Catharines&DGUIDlist=2021A00053526053&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=barrie&DGUIDlist=2021A00053543042&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=North%20Bay&DGUIDlist=2021A00053548044&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=London&DGUIDlist=2021A00053539036&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Stratford&DGUIDlist=2021A00053531011&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Brantford&DGUIDlist=2021A00053529006&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Trenton&DGUIDlist=2021A00053512015&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Belleville&DGUIDlist=2021A00053512005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Brooklin&DGUIDlist=2021A00053518009&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Bowmanville&DGUIDlist=2021A00053518017&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Burlington&DGUIDlist=2021A00053524002&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Brighton&DGUIDlist=2021A00053514004&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Brampton&DGUIDlist=2021A00053521010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Orangeville&DGUIDlist=2021A00053522014&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Bolton&DGUIDlist=2021A00053521024&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Guelph&DGUIDlist=2021A00053523008&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=kitchener&DGUIDlist=2021A00053530013&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Hamilton&DGUIDlist=2021A00053525005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Scarborough%20Agincourt&DGUIDlist=2013A000435093&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Goderich&DGUIDlist=2021A00053540028&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Newmarket&DGUIDlist=2021A00053519048&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Oakville&DGUIDlist=2021A00053524001&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=bolton&DGUIDlist=2021S05100080&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=vaughan&DGUIDlist=2021A00053519028&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=milton&DGUIDlist=2021A00053524009&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Georgetown&DGUIDlist=2021S05100314&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Woodstock&DGUIDlist=2021A00053532042&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Chatham&DGUIDlist=2021A00053536020&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Sarnia&DGUIDlist=2021A00053538030&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Windsor&DGUIDlist=2021A00053537039&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Cambridge&DGUIDlist=2021A00053530010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=St%20Thomas&DGUIDlist=2021A00053534021&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=nobleton&DGUIDlist=2021S05101071&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Leamington&DGUIDlist=2021A00053537003&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=fort%20erie&DGUIDlist=2021A00053526003&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
]

columns = [
    "City",
    "Population, 2021",
    "Population, 2016",
    "Population percentage change, 2016 to 2021",
    "Total private dwellings",
    "Population density per square kilometre",
    "Median after-tax income in 2020 among recipients ($)",
    "Median after-tax income of household in 2020 ($)",
    "65 years and over",
    "0 to 14 years",
    "15 to 19 years",
    "20 to 24 years",
    "0 to 24 years",
    "25 to 64 years",
    "Median age of the population",
    "Owner",
    "Bachelor's degree or higher",
    "Average household size",
]

In [12]:
scrape_df = pd.DataFrame(columns=columns)

# for each url, go through each <td> and find data that matches with the columns I want

for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    data = {}
    
    data["City"] = (
        soup.find("table", class_="table")
        .find("thead")
        .find("tr")
        .find_all("th")[1]
        .get_text()
        .split("More")[0]
        .strip()
    )
    for td in soup.find_all("td"):
        for col in columns:
            if f"{col} - Counts - Total" in td.attrs["title"] and data.get(col) is None:
                data[col] = int(re.sub('\D', "", td.get_text()))
        
    scrape_df = pd.concat([scrape_df, pd.DataFrame(data, index=[0])], ignore_index=True)
    
scrape_df["Population percentage change, 2016 to 2021"] = (scrape_df["Population, 2021"] - scrape_df["Population, 2016"]) / scrape_df["Population, 2021"] * 100

# We assume linear growth to extrapolate the population
scrape_df["Population percentage change, 2016 to 2021"] = scrape_df["Population percentage change, 2016 to 2021"].apply(lambda num: round(num, 1))

scrape_df["Population, 2026"] = scrape_df["Population, 2021"] * (1 + scrape_df["Population percentage change, 2016 to 2021"] / 100)

scrape_df["0 to 24 years"] = scrape_df["0 to 14 years"] + scrape_df["15 to 19 years"] + scrape_df["20 to 24 years"]

scrape_df["25 to 64 years"] = scrape_df["Population, 2021"] - (scrape_df["65 years and over"] + scrape_df["0 to 24 years"])

# the columns values lost the decimal point when we scraped the info
scrape_df['Median age of the population'] = scrape_df['Median age of the population'] / 10
scrape_df['Average household size'] = scrape_df['Average household size'] / 10

# Remove the unnecessary columns
scrape_df.drop(['0 to 14 years', '15 to 19 years', '20 to 24 years'], axis=1, inplace=True)

# remove everything after the comma and any special characters
scrape_df['City'] = scrape_df['City'].apply(lambda x: x.split(",")[0])
scrape_df['City'] = scrape_df['City'].apply(lambda x: x.split("/")[0])

scrape_df['City'] = scrape_df['City'].str.replace('[^A-Za-z\s]+', '')
scrape_df['City'] = scrape_df['City'].str.replace("Greater Sudbury ", "Sudbury")

scrape_df['City'] = scrape_df['City'].str.replace("ChathamKent", "Chatham")
scrape_df['City'] = scrape_df['City'].str.replace("Quinte West", "Trenton")


C:\Users\shann\AppData\Local\Temp\ipykernel_11764\4225853498.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  scrape_df['City'] = scrape_df['City'].str.replace('[^A-Za-z\s]+', '')


In [13]:
active_city_count = pd.read_csv("combined.csv")

for index,city in active_city_count.iterrows():
    scrape_df.loc[scrape_df['City'] == city[0], 'Pharmacies'] = city[1] 
    
scrape_df["Population, 2021 / pharmacy"] = scrape_df["Population, 2021"] / scrape_df["Pharmacies"]
scrape_df["Population, 2026 / pharmacy"] = scrape_df["Population, 2026"] / scrape_df["Pharmacies"]

In [14]:
closed_city_count = pd.read_csv("closed.csv")

for index,city in closed_city_count.iterrows():
    scrape_df.loc[scrape_df['City'] == city[0], 'Closed Pharmacies'] = city[1] 


In [15]:
scrape_df.to_csv("census_canada_scrape.csv", index=False)